In [ ]:
from pycram.designators.action_designator import *
from pycram.designators.location_designator import *
from pycram.designators.object_designator import *
from pycram.failure_handling import RetryMonitor
from pycram.bullet_world import BulletWorld, Object
from pycram.language import Monitor, Code
from pycram.pose import Pose
from pycram.process_module import simulated_robot
from pycram.ros.viz_marker_publisher import VizMarkerPublisher

In [ ]:
world = BulletWorld("DIRECT")
viz = VizMarkerPublisher()
robot = Object("pr2", "robot", "pr2.urdf", pose=Pose([1, 2, 0]))
apartment = Object("apartment", "environment", "apartment.urdf")

In [ ]:
robot_desig = BelieveObject(names=["pr2"])
apartment_desig = BelieveObject(names=["apartment"])

In [ ]:
height = 0.3
sleep_time = 2

def monitor():
    global height
    time.sleep(1)
    if height == 0.3:
        height = 0.1
    else:
        height = 0.3
    print("Triggered")
    return True


In [ ]:
with simulated_robot:
    sleep = Code(lambda: time.sleep(sleep_time))
    code = Code(lambda: print(height))

    # Call MoveTorso like this to be able to adjust the parameters manually between repeat steps, for example
    # the height for MoveTorsoAction, or the object one wants to grasp for pickup
    move_torso = Code(lambda: MoveTorsoAction([height]).resolve().perform())

    # plan = print out current height, then move torso, then sleep for sleep_time. Do that twice.
    # If monitor condition is triggered (aka returns True), current action is interrupted, and plan is repeated
    plan = (code + move_torso + sleep) * 2 >> Monitor(monitor)

    RetryMonitor(plan, max_tries=5).perform()